In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import time
import os
import os.path as path
import copy

from gait_analysis import AnnotationsCasia as Annotations
from gait_analysis import CasiaDataset
from gait_analysis.Config import Config
from gait_analysis import Composer
from gait_analysis import WeightWatcher
from gait_analysis import AccuracyTrackerTrainTest

import matplotlib.pyplot as plt

from gait_analysis.Models import PretrainConvFlow
from gait_analysis.Models.TransferConvLSTMFlow import TransferConvLSTMFlow

print("done")

done


In [11]:
BATCH_SIZE = 1
TIME_STEPS = 40
SLICE_FROM_TIME_STEP = 0 #slices from timestep SLICE_FROM_TIMESTEP to the last one

In [12]:
net = TransferConvLSTMFlow()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net.to(device)

net.load_state_dict(torch.load('/home/ron/PycharmProjects/Gait2019/gait_project/saved_models/TransferConvLSTMFlow/TransferConvLSTMFlow.pth'))

TODO: doesn't x = x.view(BATCH_SIZE,TIME_STEPS*LSTM_HIDDEN_FEATURES) mix up batch size order?
TODO: BATCH_SIZE is currently fixed and one
TODO: Is x_arr in device?
cuda:0


In [13]:
net.eval()

TransferConvLSTMFlow(
  (conv_net): PretrainConvFlow(
    (features): Sequential(
      (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
      (4): ReLU()
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
      (7): ReLU()
      (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (9): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
      (10): ReLU()
      (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (12): Conv2d(16, 6, kernel_size=(3, 3), stride=(1, 1))
      (13): ReLU()
      (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (classifier): Sequential(
      (0): Linear(in_features=90, out_features=120, bias

In [14]:
c = Config()
c.config['indexing']['grouping'] = 'person_sequence_angle'

composer = Composer()
transformer = composer.compose()
dataset = CasiaDataset(transform=transformer)

In [15]:
dataset_size = len(dataset)
indices = list(range(dataset_size))
print("Dataset size:", len(dataset))
#print("Indices size:", len(indices))
#split = int(np.floor(VALIDATION_SPLIT * dataset_size))
#print("Split:", split)
#if SHUFFLE_DATASET:
#    np.random.seed(RANDOMIZED_SEED)
#    np.random.shuffle(indices)
#train_indices, test_indices = indices[split:], indices[:split]
#train_sampler = torch.utils.data.SequentialSampler(train_indices)
test_sampler = torch.utils.data.SequentialSampler(indices)

#train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler)

Dataset size: 45


In [16]:
inputs, labels = dataset[0]
data_in = [s.to(device) for s in inputs['flows']]
labels = labels.to(device)
